# Crashes on a Street

In [1]:
import pytz
import folium
from folium.plugins import HeatMap

import pandas as pd
from datetime import datetime, timedelta
from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
start_date = pytz.timezone('America/Denver').localize(datetime(2013,1,1))
end_date = pytz.timezone('America/Denver').localize(datetime(2023,3,1))
most_recent_crash = cda.most_recent_crash_timestamp()

In [4]:
query = f"""
select
sr.gid
, sr.lrsroute
, st_length(sr.geom_denver) / 5280 as length_miles
, c.sbi
, c.fatality
, c.pedestrian_ind
, c.bicycle_ind
, c.geo_lon
, c.geo_lat
, c.reported_date
, c.reported_date at time zone 'America/Denver' as reported_date_local
, c.incident_address_corrected

from street_routes sr
inner join crashes c on st_dwithin(sr.geom_denver, c.geom_denver, 25)

where sr.gid = 1392
and reported_date at time zone 'America/Denver' > '{start_date.strftime('%Y-%m-%d')}'
and reported_date at time zone 'America/Denver' < '{end_date.strftime('%Y-%m-%d')}'
and not at_freeway

order by reported_date desc
"""

street_crashes = pd.read_sql(query, cda.conn)

In [5]:
street_crashes.groupby(street_crashes.reported_date_local.dt.year).agg(
    crashes=('reported_date_local', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date_local,,,,,
2013,17,0,0,1,0
2014,17,0,2,2,1
2015,16,0,0,0,1
2016,14,0,0,0,1
2017,21,0,1,0,1
2018,15,0,0,1,1
2019,34,0,1,8,0
2020,15,0,0,1,2
2021,11,0,1,1,0


In [6]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(street_crashes[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [7]:
days_in_data = (end_date - start_date).days

In [8]:
days_between_crashes = days_in_data / len(street_crashes)
days_between_crashes

21.085227272727273

In [9]:
crashes_per_mile_per_week = (len(street_crashes) / street_crashes.iloc[0].length_miles) / (days_in_data/7)
crashes_per_mile_per_week

0.4790477314103727

In [10]:
street_crashes.sort_values(by='geo_lat')

,gid,lrsroute,length_miles,sbi,fatality,pedestrian_ind,bicycle_ind,geo_lon,geo_lat,reported_date,reported_date_local,incident_address_corrected
87,1392,GILPIN ST4,0.693012,False,False,0.0,0.0,-104.967247,39.736826,2018-05-28 19:59:00+00:00,2018-05-28 13:59:00.000000,E 13TH AVE / N GILPIN ST
41,1392,GILPIN ST4,0.693012,False,False,0.0,0.0,-104.967247,39.736826,2020-01-05 17:27:00+00:00,2020-01-05 10:27:00.000000,E 13TH AVE / N GILPIN ST
47,1392,GILPIN ST4,0.693012,False,False,0.0,0.0,-104.967247,39.736826,2019-11-03 01:57:00.000001+00:00,2019-11-02 19:57:00.000001,E 13TH AVE / N GILPIN ST
54,1392,GILPIN ST4,0.693012,False,False,0.0,0.0,-104.967247,39.736826,2019-07-31 16:47:00+00:00,2019-07-31 10:47:00.000000,E 13TH AVE / N GILPIN ST
63,1392,GILPIN ST4,0.693012,False,False,0.0,0.0,-104.967247,39.736826,2019-03-28 03:57:00+00:00,2019-03-27 21:57:00.000000,E 13TH AVE / N GILPIN ST
...,...,...,...,...,...,...,...,...,...,...,...,...
17,1392,GILPIN ST4,0.693012,False,False,0.0,0.0,-104.967211,39.744818,2021-12-07 01:33:00+00:00,2021-12-06 18:33:00.000000,N GILPIN ST / E 18TH AVE
112,1392,GILPIN ST4,0.693012,False,False,0.0,0.0,-104.967211,39.744818,2016-12-18 17:00:00+00:00,2016-12-18 10:00:00.000000,E 18TH AVE / N GILPIN ST
3,1392,GILPIN ST4,0.693012,False,False,0.0,0.0,-104.967212,39.746080,2023-02-02 00:03:00+00:00,2023-02-01 17:03:00.000000,E 19TH AVE / N GILPIN ST
68,1392,GILPIN ST4,0.693012,False,False,0.0,0.0,-104.967212,39.746080,2019-02-27 00:53:00+00:00,2019-02-26 17:53:00.000000,E 19TH AVE / N GILPIN ST
